In [1]:
%load_ext Cython

In [13]:
%%cython
import numpy as np
import pandas as pd
#dictionary型に変換する関数
def two_list_to_dict(key_list, val_list):
      return dict(zip(key_list, val_list))

#csvを読み込んで行ごとに分割
def read_file(filename):
    with open(str(filename),'r',encoding='utf8') as fbrand:
        txt=fbrand.read() 
        lines = txt.split("\n")
        return lines

#csvの各行を分割してコーデごとのブランドlistのlistを作る.
def make_Coordinate(lines):
    Coordinate=[]
    #重複のない辞書にするためにsetを用いる
    s=set()
    for line in lines:
        brands=line.split(",")
        Coordinate.append(brands)
    return Coordinate

#good_tagsのみに絞る
def make_goodtags(lines,good_tags_set):
    good_tags_list=[]
    #重複のない辞書にするためにsetを用いる
    s=set()
    s=good_tags_set
    for line in lines:
        tags=line.split(",")
        good_tags=[t for t in tags if t in s] 
        #if len(good_tags) == 0:
        good_tags_list.append(good_tags)
    return good_tags_list

def make_brandset(lines):
    #重複のない辞書にするためにsetを用いる
        s=set()
        for line in lines:
            brands=line.split(",")
            for brand in brands:
                s.add(brand)
        brand_set=list(s)
        print(len(brand_set))
        return brand_set

def make_dict(brandset):
    #dictionaryのindex
    index=[]
    for i in range(len(brandset)):
        index.append(i)
    #辞書
    brand_dict=two_list_to_dict(brandset,index)
    return brand_dict

def brand_to_dictnum(Coordinates,brand_dict):
    W = [] # words
    #ブランドと補助情報の文書ごとのリストを作る
    for Coordinate in Coordinates:
    #list()をつけない場合iterator objectが帰ってくる
        W.append(list(map(lambda v:brand_dict[v],Coordinate)))
    return W

#結果に関する関数  
def brand_to_topic(Coordinates,nctm_Z): #nctm.Zはnctm.fitの結果が入っている
    brand_topic=[]
    for brands, Topics in zip(Coordinates,nctm_Z):
        for brand,Topic in zip(brands,Topics):
            pair=[brand,Topic]
            brand_topic.append(pair)
    return brand_topic

def brand_topic_list_f(brand_topic,K):
    brand_topic_list=[]
    for i in range(K):
        aaa=[x for x in brand_topic if x[1] == i]
        #print(aaa)
        brand_topic_list.append(aaa)
    return brand_topic_list

def t_setlist_brand_f(brand_topic_list,K):
    #brandについて
    t_setlist_brand=[] #各トピックごとのbrandのsetを代入するリスト
    for i in range(K): #各トピックで回す
        a=[] #各トピックでsetを作るためのローカル変数
        for item in brand_topic_list[i]:
            a.append(item[0])
        a=set(a) #セットにする
        a=list(a) #リストに直す
        t_setlist_brand.append(a) #現在のトピックのsetをグローバルのリストに追加
    #print(t_setlist[0]) 
    return t_setlist_brand

def count_topic_f(brand_topic_list,t_setlist_brand,K):
    count_topic=[] #全トピックの出現回数リストのリスト
    for i in range(K): #トピックループ
        count=np.zeros((len(t_setlist_brand[i]))) #今見ているトピックのsetの長さ分のbrandカウント配列を初期化
        for item in brand_topic_list[i]: #itemはリスト
            count[t_setlist_brand[i].index(item[0])]+=1 #今見ているbrandのインデックスをカウントアップ
        count_topic.append(count) #一つのトピックについて見終えたらグローバルに追加
    return count_topic

def count_noise(tag_set,R):
    count=np.zeros(len(tag_set)) #今見ているトピックのsetの長さ分のbrandカウント配列を初期化
    for item in R: #itemはリスト
        count[item]+=1 #今見ているbrandのインデックスをカウントアップ
    return count

def write_result(filename,t_setlist_brand,count_topic):
    topic_num=0
    for item,count in zip(t_setlist_brand,count_topic):
        tuples=[]
        for b,c in zip(item,count):
            tuples.append(tuple((b,c)))
        tuples.sort(key=lambda x:x[1],reverse=True) #x[1]に出現回数が入っているので多い順にソート
        df=pd.DataFrame(tuples)
        if topic_num < 10:
            df.to_csv( './cleansing_data/result_iter1000/topic_75/topic'+'0'+str(topic_num)+str(filename), index=False )
        else:
            df.to_csv( './cleansing_data/result_iter1000/topic_75/topic'+str(topic_num)+str(filename), index=False )
        topic_num+=1
    print("END")

In [3]:
good_tag_lines=read_file('./cleansing_data/goodtags.csv')
#good_tag_input=make_Coordinate(good_tag_lines)
good_tag_input=make_Coordinate(good_tag_lines[0:10000])
Coordinate_lines=read_file('./cleansing_data/goodCoords.csv')
# good_Coordinate_input=make_Coordinate(Coordinate_lines)
good_Coordinate_input=make_Coordinate(Coordinate_lines[0:10000])

In [5]:
# モデルに入れる前に形式を合わせる必要がある
tag_set=make_brandset(good_tag_lines[0:10000])
# tag_set=make_brandset(good_tag_lines)
tag_dict=make_dict(tag_set)
brand_set=make_brandset(Coordinate_lines[0:10000])
# brand_set=make_brandset(Coordinate_lines)
brand_dict=make_dict(brand_set)
tag_W=brand_to_dictnum(good_tag_input,tag_dict)
Coordinate_W=brand_to_dictnum(good_Coordinate_input,brand_dict)

3218
3297


In [6]:
print(good_tag_input[0:10])
print(tag_W[0:10])

[["who's who chico", 'シンプル', 'フレアスカート', 'ニットスカート'], ['シンプル', 'スニーカー', 'タートルネックニット', 'ロシアン帽', '青文字系', 'ニコアンド', 'コンバース', '冬'], ['WEGO', '出勤コーデ', 'ヘアバンド', 'お団子ヘアー', 'レーストップス', 'カーディガン', 'リボン', 'コンバース', 'GU'], ['冬', 'ワンピース', '白ニット', 'ダッフルコート', 'ミニスカート', 'ヘアバンド'], ['結婚式', 'パーティースタイル', 'オケージョン', 'オールインワン', 'サロペット', 'ダウンジャケット', 'ネックレス', 'きれいめカジュアル'], ['きれいめカジュアル', 'カジュアル', 'ホワイト', 'ベージュ', 'カットオフ', 'キャップ', 'ストリート', 'スウェット', 'ローキャップ', 'ラフ'], ['アウター', 'ピンク', 'ノーカラーコート', 'シャギー', 'ワイドスリーブ', 'コーデュロイ', 'ニット'], ['ダメージデニムパンツ', 'レオパード', 'ショートブーツ', 'Doシンプル部', 'いい友連合', 'ロングカーディガン', 'オトナカジュアル', 'きれいめカジュアル', 'お団子ヘアー', 'ダメージデニム'], ['ミリタリー', 'ミリタリージャケット', 'フライトジャケット', '黒スキニー', 'ユニクロ', 'ドレスシューズ', '白ソックス', '冬'], ['シンプル', 'ダル着', 'チャンピオン', 'AVIREX', 'ディッキーズ(Dickies)', 'ちょっとそこまで']]
[[1466, 1921, 815, 2737], [1921, 1907, 1348, 20, 2413, 1713, 326, 3054], [2042, 1681, 228, 1121, 1561, 1131, 583, 326, 403], [3054, 2669, 1477, 2253, 492, 228], [2342, 589, 615, 65, 284, 721, 327, 2696], [2696, 2548, 1460, 1373, 1655, 

In [8]:
from nctm import NCTM
W=Coordinate_W
X=tag_W
Vw = len(W)
Vx = len(X)
K = 75
alpha = 0.1
beta= 0.1
gamma= 0.1
eta = 1.0
max_iter = 1000
nctm = NCTM(K=K, alpha=alpha, beta=beta, gamma=gamma, eta=eta, max_iter=max_iter)

In [9]:
import time
start = time.time()
nctm.fit(W,X,Vw,Vx)
elapsed_time = time.time() - start
print("elapsed_time:{0}".format(elapsed_time))

The installed widget Javascript is the wrong version.


実行中...
実行中...
実行中...
実行中...
実行中...
実行中...
実行中...
実行中...
実行中...
実行中...
elapsed_time:3605.6753101348877


In [ ]:
## 改めて普通のLDAをやってみるのもあり

In [10]:
brand_topic=brand_to_topic(good_Coordinate_input,nctm.Z)
tag_topic=brand_to_topic(good_tag_input,nctm.Y)

#topicごとにブランドとトピックの組をまとめる
brand_topic_list=brand_topic_list_f(brand_topic,K)
tag_topic_list=brand_topic_list_f(tag_topic,K)

#トピックごとに出現ブランドのセットを作る
t_setlist_brand=t_setlist_brand_f(brand_topic_list,K)
t_setlist_tag=t_setlist_brand_f(tag_topic_list,K)

# 各トピックごとのサイズの確認
TopicBrandSize=[]
TopicTagSize=[]
index=[]
for i in range(len(t_setlist_brand)):
    TopicBrandSize.append(len(t_setlist_brand[i]))
    TopicTagSize.append(len(t_setlist_tag[i]))
    index.append(i)
print(TopicBrandSize)
print(TopicTagSize)

[129, 118, 134, 114, 119, 129, 114, 120, 130, 86, 119, 118, 100, 119, 142, 130, 135, 128, 111, 124, 139, 133, 154, 112, 125, 119, 111, 110, 135, 124, 105, 113, 110, 142, 128, 115, 125, 119, 122, 114, 105, 115, 121, 131, 132, 110, 139, 124, 114, 124, 115, 126, 131, 132, 127, 125, 140, 116, 148, 134, 123, 130, 132, 131, 119, 135, 118, 122, 116, 117, 125, 146, 123, 111, 148]
[407, 332, 334, 399, 464, 355, 327, 341, 407, 580, 400, 334, 363, 458, 337, 357, 338, 381, 345, 322, 396, 334, 355, 440, 365, 337, 346, 457, 343, 369, 391, 396, 506, 361, 336, 345, 355, 331, 372, 353, 350, 353, 363, 322, 350, 413, 325, 425, 434, 465, 310, 333, 429, 314, 368, 346, 360, 356, 309, 360, 343, 350, 356, 336, 443, 322, 364, 367, 346, 346, 336, 452, 351, 346, 393]


In [11]:
print(nctm.R[0:100])

[array([0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0]), array([0]), array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0]), array([0]), array([0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0]), array([0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0]), array([0, 0, 0]), 

In [12]:
#セットを用いて出現回数をカウント
count_topic_brand=count_topic_f(brand_topic_list,t_setlist_brand,K)
count_topic_tag=count_topic_f(tag_topic_list,t_setlist_tag,K)

In [14]:
#結果の書き込み
import pandas as pd
write_result('brands.csv',t_setlist_brand,count_topic_brand)
write_result('tags.csv',t_setlist_tag,count_topic_tag)

END
END


In [15]:
## ノイズと判断されたものについて
noise_list=[]
for r,tag in zip(nctm.R,X):
    for r0, tag0 in zip(r,tag) :
        print(r0,tag0)
        if(r0==0):
            noise_list.append(tag0)

0 1466
0 1921
0 815
0 2737
0 1921
0 1907
0 1348
0 20
0 2413
0 1713
0 326
0 3054
0 2042
0 1681
0 228
0 1121
0 1561
0 1131
0 583
0 326
0 403
0 3054
0 2669
0 1477
0 2253
0 492
0 228
0 2342
0 589
0 615
0 65
0 284
0 721
0 327
0 2696
0 2696
0 2548
0 1460
0 1373
0 1655
0 1136
0 2060
0 1643
0 2916
0 731
0 1616
0 2292
0 2518
0 2124
0 2362
0 2010
0 3075
0 1509
0 3165
0 461
0 1734
0 426
0 2088
0 457
0 2696
0 1121
0 1716
0 2831
0 2161
0 2793
0 1590
0 3003
0 580
0 2301
0 3054
0 1921
0 514
0 2235
0 1835
0 1967
0 1349
0 1921
0 2963
0 1098
0 3075
0 1223
0 319
0 829
0 566
0 1600
0 1506
0 2696
0 2026
0 403
0 515
0 850
0 133
0 1921
0 2184
0 3054
0 2696
0 2546
0 1590
0 2548
0 1945
0 0
0 562
0 2328
0 2719
0 1244
0 1142
0 1921
0 539
0 333
0 800
0 2702
0 3054
0 2696
0 2771
0 2776
0 2284
0 1509
0 26
0 1921
0 405
0 1345
0 188
0 2253
0 636
0 3054
0 2696
0 2546
0 68
0 2474
0 2547
0 314
0 716
0 117
0 2001
0 403
0 320
0 3085
0 2834
0 2652
0 733
0 1921
0 136
0 1921
0 1907
0 539
0 1745
0 2963
0 721
0 2499
0 1512
0 9

0 3085
0 3054
0 3003
0 1262
0 890
0 34
0 3075
0 1373
0 1241
0 2548
0 1084
0 1921
0 1907
0 2963
0 68
0 1262
0 2967
0 892
0 1321
0 864
0 1727
0 1838
0 2286
0 463
0 37
0 1485
0 1223
0 1934
0 1262
0 1579
0 49
0 1223
0 2597
0 126
0 1774
0 2044
0 1241
0 1921
0 457
0 539
0 2292
0 2088
0 1258
0 2776
0 1668
0 2871
0 1241
0 1974
0 314
0 2762
0 850
0 2714
0 1074
0 2639
0 1044
0 2305
0 731
0 2771
0 989
0 1136
0 991
0 2353
0 3054
0 2548
0 3054
0 2547
0 2044
0 2693
0 254
0 2119
0 2776
0 800
0 1073
0 1048
0 1921
0 1131
0 2858
0 320
0 2967
0 2705
0 806
0 1781
0 484
0 580
0 2837
0 1623
0 2837
0 1921
0 745
0 1348
0 242
0 266
0 2119
0 889
0 548
0 638
0 1974
0 326
0 1627
0 1883
0 1223
0 2423
0 1527
0 2646
0 2226
0 2564
0 1070
0 2669
0 1357
0 329
0 3054
0 850
0 3060
0 2883
0 1590
0 2673
0 1546
0 2132
0 1136
0 913
0 3003
0 1921
0 539
0 3054
0 1745
0 2963
0 2027
0 561
0 1392
0 3075
0 2883
0 1945
0 156
0 1171
0 1921
0 3003
0 2963
0 400
0 3085
0 2510
0 133
0 3095
0 1262
0 1536
0 1921
0 1907
0 539
0 1348
0 3054

0 600
0 285
0 2060
0 1136
0 1244
0 3075
0 649
0 403
0 3054
0 3075
0 2547
0 2474
0 2719
0 2548
0 1879
0 68
0 1611
0 2599
0 1136
0 539
0 1907
0 3155
0 1182
0 889
0 1392
0 1241
0 2364
0 3070
0 808
0 1671
0 3054
0 2546
0 3080
0 1608
0 3054
0 1907
0 1934
0 969
0 3075
0 1483
0 2664
0 1244
0 2095
0 380
0 3054
0 1315
0 3003
0 1472
0 2226
0 296
0 2301
0 333
0 2382
0 3054
0 1945
0 639
0 890
0 457
0 2548
0 1921
0 1262
0 68
0 2010
0 3054
0 2027
0 1907
0 2548
0 539
0 1921
0 2147
0 2702
0 1223
0 2353
0 2432
0 1108
0 1223
0 2597
0 1973
0 80
0 2603
0 1453
0 1136
0 911
0 1509
0 1612
0 1588
0 2181
0 2008
0 3014
0 2336
0 457
0 2548
0 1921
0 3054
0 457
0 1907
0 1609
0 415
0 3054
0 3150
0 1582
0 1241
0 2263
0 1250
0 18
0 1509
0 1294
0 2250
0 2301
0 2776
0 1570
0 457
0 2548
0 1921
0 2967
0 1720
0 2486
0 889
0 1512
0 2618
0 319
0 2133
0 3054
0 850
0 649
0 572
0 2825
0 2631
0 336
0 1907
0 2548
0 457
0 1774
0 731
0 2963
0 2027
0 88
0 1907
0 3054
0 2866
0 2092
0 2771
0 720
0 594
0 2776
0 1472
0 320
0 2837
0 136

0 2754
0 457
0 1921
0 1915
0 3054
0 1241
0 403
0 2353
0 1590
0 989
0 2137
0 399
0 301
0 2619
0 1720
0 548
0 3211
0 2382
0 1048
0 3003
0 1434
0 1358
0 1400
0 1271
0 1108
0 3054
0 2548
0 2077
0 845
0 3054
0 2010
0 2547
0 2510
0 845
0 2060
0 457
0 1223
0 2597
0 1921
0 2119
0 548
0 2794
0 996
0 3112
0 1745
0 2479
0 603
0 900
0 1627
0 1108
0 863
0 800
0 1907
0 3054
0 2546
0 539
0 800
0 457
0 863
0 3023
0 1241
0 1570
0 1643
0 1820
0 660
0 2524
0 811
0 2548
0 2038
0 2119
0 548
0 600
0 1244
0 1945
0 2782
0 3160
0 889
0 3054
0 3060
0 2719
0 2824
0 568
0 1915
0 1940
0 1241
0 1509
0 1407
0 2364
0 1348
0 3054
0 1934
0 457
0 2211
0 2824
0 1131
0 2548
0 2447
0 2546
0 2217
0 3178
0 2696
0 3132
0 1955
0 2382
0 1774
0 3075
0 2391
0 1086
0 2010
0 2547
0 2347
0 1727
0 1941
0 2639
0 411
0 2253
0 336
0 539
0 326
0 2548
0 1907
0 539
0 1333
0 2280
0 2546
0 3054
0 373
0 2181
0 1434
0 1071
0 133
0 2301
0 854
0 1384
0 755
0 2837
0 3054
0 457
0 1907
0 1392
0 2547
0 403
0 2776
0 1278
0 3075
0 1720
0 548
0 969
0 3

0 3054
0 2673
0 1425
0 3075
0 539
0 2548
0 319
0 3126
0 2771
0 2913
0 3054
0 195
0 2298
0 551
0 2599
0 1745
0 1907
0 3158
0 1262
0 1933
0 400
0 1373
0 2546
0 68
0 2027
0 703
0 974
0 2510
0 2441
0 2344
0 1626
0 1907
0 539
0 2548
0 1945
0 1940
0 2546
0 1232
0 3054
0 316
0 457
0 3054
0 3072
0 3075
0 1983
0 969
0 1720
0 548
0 1883
0 1048
0 1545
0 2788
0 2913
0 3054
0 2548
0 457
0 1907
0 3075
0 2547
0 2108
0 1907
0 3075
0 326
0 566
0 2027
0 2226
0 986
0 1682
0 3054
0 3054
0 457
0 2548
0 1907
0 284
0 1241
0 1671
0 2425
0 2518
0 3075
0 539
0 1921
0 1781
0 2696
0 1223
0 1509
0 457
0 2382
0 99
0 863
0 539
0 2548
0 2546
0 68
0 1745
0 2673
0 3054
0 2696
0 457
0 1907
0 3075
0 539
0 326
0 136
0 39
0 986
0 1223
0 2597
0 2364
0 926
0 391
0 1003
0 1315
0 562
0 1682
0 539
0 989
0 2510
0 974
0 2027
0 2858
0 731
0 1907
0 1244
0 1136
0 1940
0 1151
0 2060
0 2027
0 2226
0 822
0 3054
0 484
0 2108
0 2095
0 2253
0 59
0 414
0 2673
0 2724
0 1250
0 1695
0 484
0 1907
0 326
0 2775
0 1315
0 986
0 1682
0 1223
0 2597


...

0 1546
0 3098
0 1742
0 2533
0 3105
0 136
0 3085
0 1590
0 1241
0 3075
0 1945
0 2696
0 2548
0 1921
0 1915
0 136
0 3085
0 1419
0 1268
0 2568
0 386
0 2985
0 1131
0 816
0 2858
0 2353
0 2044
0 1036
0 811
0 2170
0 3075
0 539
0 373
0 2027
0 974
0 2077
0 2010
0 1980
0 461
0 2284
0 3003
0 1805
0 2042
0 2027
0 731
0 2967
0 1315
0 320
0 2119
0 323
0 1748
0 1221
0 1241
0 2076
0 2467
0 2788
0 1446
0 3050
0 2644
0 2302
0 373
0 3128
0 2510
0 1582
0 1588
0 806
0 2384
0 1258
0 2384
0 1675
0 1921
0 2548
0 2253
0 68
0 815
0 2816
0 1126
0 2731
0 1607
0 1223
0 2597
0 2439
0 783
0 400
0 1270
0 3127
0 136
0 1945
0 2026
0 1611
0 403
0 850
0 91
0 1371
0 136
0 2027
0 3085
0 2060
0 325
0 322
0 1747
0 2639
0 2390
0 136
0 3085
0 2673
0 126
0 1945
0 1643
0 2384
0 2963
0 2967
0 2353
0 3003
0 2910
0 539
0 403
0 731
0 2042
0 2161
0 1915
0 2963
0 136
0 3076
0 326
0 2247
0 2119
0 3075
0 1315
0 539
0 2776
0 2027
0 2858
0 731
0 1618
0 974
0 1618
0 539
0 1915
0 1320
0 403
0 2639
0 2599
0 1482
0 1392
0 2939
0 996
0 3112
0 29

0 1915
0 2819
0 254
0 136
0 1940
0 2077
0 543
0 1907
0 2221
0 1136
0 133
0 3054
0 2364
0 974
0 3085
0 2808
0 2935
0 3057
0 1333
0 2292
0 2133
0 2060
0 822
0 1294
0 2132
0 338
0 1238
0 1203
0 68
0 1897
0 2202
0 567
0 430
0 2301
0 2942
0 1378
0 2042
0 893
0 1036
0 1915
0 1840
0 3211
0 650
0 2669
0 900
0 2381
0 1087
0 2913
0 2119
0 2963
0 1048
0 1182
0 491
0 1136
0 419
0 3142
0 539
0 1036
0 1032
0 649
0 1142
0 1980
0 3076
0 1842
0 1130
0 2388
0 2567
0 2910
0 2763
0 2425
0 1565
0 1974
0 2547
0 1992
0 940
0 806
0 2344
0 1967
0 1485
0 2353
0 136
0 2060
0 2548
0 906
0 766
0 1136
0 2441
0 2447
0 68
0 1477
0 2837
0 642
0 2027
0 2963
0 974
0 1224
0 461
0 1781
0 1232
0 619
0 3057
0 1965
0 1315
0 806
0 1590
0 605
0 2133
0 1232
0 2027
0 2108
0 974
0 2858
0 2547
0 2090
0 2963
0 527
0 137
0 1198
0 2963
0 731
0 2353
0 539
0 2858
0 600
0 3080
0 1398
0 2702
0 83
0 2704
0 1241
0 1048
0 3211
0 2548
0 1244
0 3101
0 1782
0 136
0 2027
0 1590
0 2060
0 721
0 2613
0 1434
1 1927
0 2719
0 3075
0 2875
0 2353
0 271

0 2286
0 127
0 2776
0 780
0 1907
0 3075
0 1098
0 2548
0 1921
0 2292
0 1151
0 68
0 3054
0 457
0 2963
0 893
0 1340
0 67
0 815
0 1477
0 1348
0 2285
0 2520
0 2105
0 2307
0 2729
0 746
0 1913
0 2027
0 2963
0 974
0 539
0 563
0 2353
0 989
0 1907
0 731
0 1223
0 539
0 1460
0 2027
0 731
0 2441
0 745
0 2837
0 3
0 1618
0 2512
0 2027
0 2963
0 974
0 619
0 2963
0 974
0 1098
0 93
0 1644
0 1975
0 3085
0 642
0 2669
0 1036
0 2548
0 1921
0 320
0 1086
0 2219
0 2441
0 2963
0 460
0 136
0 2027
0 1973
0 527
0 137
0 2645
0 1766
0 464
0 93
0 2353
0 2119
0 3076
0 1915
0 1114
0 1883
0 2912
0 2100
0 364
0 136
0 2518
0 1086
0 68
0 1223
0 769
0 2794
0 7
0 1027
0 1043
0 1223
0 327
0 1512
0 2631
0 1992
0 2027
0 436
0 731
0 974
0 1948
0 2858
0 2548
0 1315
0 1224
0 136
0 2027
0 3178
0 3054
0 2963
0 457
0 2858
0 2044
0 2364
0 2540
0 2060
0 872
0 373
0 2027
0 3054
0 693
0 2384
0 2027
0 1742
0 3199
0 2347
0 561
0 2963
0 1618
0 1643
0 1921
0 1945
0 136
0 3085
0 2673
0 3054
0 1915
0 2834
0 1590
0 1727
0 3075
0 721
0 2548
0 136

0 2025
0 1293
0 2233
0 1913
0 1609
0 124
0 2088
0 199
0 2180
0 2014
0 136
0 2060
0 1417
0 564
0 524
0 336
0 136
0 2979
0 1745
0 1274
0 3054
0 745
0 2858
0 123
0 2027
0 136
0 457
0 2546
0 1392
0 327
0 2669
0 1768
0 1651
0 2839
0 2342
0 2673
0 1897
0 840
0 1618
0 2027
0 2044
0 974
0 1727
0 2762
0 1992
0 2138
0 3105
0 1599
0 2060
0 136
0 2027
0 2079
0 1262
0 2548
0 1921
0 566
0 2883
0 2027
0 1787
0 2963
0 1811
0 1393
0 404
0 2250
0 1047
0 1294
0 3075
0 2353
0 3075
0 2831
0 136
0 2027
0 2226
0 2628
0 1348
0 1590
0 457
0 3075
0 1781
0 136
0 2027
0 1745
0 2673
0 561
0 2696
0 1320
0 457
0 2060
0 136
0 2027
0 3075
0 2669
0 1921
0 2425
0 461
0 566
0 136
0 320
0 2368
0 3054
0 136
0 2027
0 649
0 2866
0 1036
0 373
0 2060
0 136
0 2027
0 2831
0 2108
0 969
0 2226
0 1546
0 136
0 2963
0 539
0 1241
0 3075
0 2548
0 2253
0 2631
0 136
0 2027
0 811
0 2696
0 1320
0 457
0 2669
0 1921
0 1546
0 1460
0 136
0 2027
0 2185
0 2177
0 3054
0 3204
0 2027
0 2963
0 974
0 2696
0 2250
0 68
0 2147
0 326
0 1907
0 2060
0 136


0 539
0 1671
0 2425
0 2710
0 2438
0 2010
0 1136
0 663
0 326
0 136
0 2027
0 2353
0 2628
0 1320
0 2548
0 1727
0 2547
0 2088
0 1536
0 2963
0 893
0 2027
0 2696
0 2737
0 1315
0 136
0 2027
0 2060
0 1973
0 2108
0 472
0 2830
0 2547
0 3085
0 126
0 1921
0 1136
0 2027
0 136
0 2027
0 1590
0 3085
0 1320
0 2696
0 757
0 1097
0 2942
0 2211
0 2253
0 1570
0 68
0 757
0 2262
0 1477
0 1536
0 2027
0 2963
0 2696
0 1618
0 974
0 1907
0 850
0 1921
0 1136
0 2060
0 1476
0 136
0 3085
0 3191
0 1223
0 2027
0 2963
0 2696
0 2253
0 136
0 2785
0 1091
0 1570
0 2027
0 2963
0 974
0 539
0 1745
0 403
0 67
0 1643
0 1907
0 1921
0 136
0 1671
0 2027
0 2696
0 2524
0 2301
0 3075
0 2673
0 2087
0 2262
0 2963
0 3142
0 2119
0 2858
0 889
0 1241
0 1946
0 2088
0 1921
0 3075
0 863
0 2548
0 2027
0 2368
0 2547
0 457
0 2353
0 136
0 1036
0 2026
0 2831
0 1136
0 1579
0 845
0 1108
0 3085
0 342
0 1223
0 2801
0 2997
0 223
1 685
0 126
0 1223
0 3085
0 2866
0 1643
0 2548
0 1921
0 1219
0 2540
0 619
0 1940
0 2546
0 1626
0 2027
0 415
0 2221
0 3165
0 296

In [16]:
print(len(set(noise_list)))

3144


In [24]:
count=0
for n in noise_list:
    if n==3477:
        count+=1
print(count)

608


In [ ]:
tag_set_10000=make_brandset(good_tag_lines[0:10000])
tag_dict_10000=make_dict(tag_set_10000)
brand_set_10000=make_brandset(Coordinate_lines[0:10000])
brand_dict_10000=make_dict(brand_set_10000)